# Introduction/Business Problem 

In this project, I aim to provide suggestions to the contractor about where to open a new restaurant in New York City. It is essential for people to select a good place for food business to avoid losing money. We will use the New York City Foursquare location data to help the decision-making. We will take several factors into account, such as common venues and venue types. 

# Data

We will download the New York City Foursquare location data, same as the dataset we used in our lab session. We will get the venue names, venue types, neighborhood, Borough, and other related information in the dataset. 

# Methodology

I will use the geopy.geocoders, folium to get the neighborhood, venue, and other information of New York City, and then performed clustering of the venues in the selected borough. The clustering can tell us the distribution of each type of venues and imply the popular places for stating a new food business. I will focus on the restaurant-related venues and then give suggestions for the people who want ot open a new restaurant nearby. 

# Results

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-64::numba==0.43.1=py37h6440ff4_0
  - defaults/osx-64::anaconda==2019.03=py37_0
done

## Package Plan ##

  environment location: /Users/shuaihu/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _anaconda_depends-2019.03  |           py37_0           5 KB
    anaconda-custom            |           py37_1           3 KB
    ca-certificates-2019.9.11  |       hecc5488_0         143 KB  conda-forge
    certifi-2019.9.11          |           py37_0         147 KB  conda-forge
    openssl-1.1.1c             |       h01d97ff_0         1.9 MB  conda-forge
    tbb-2019.8                 |       h770b8ee_0         157 KB  conda-forge
    -----------------------------------------------

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


### download the New York City data

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neighborhoods_data = newyork_data['features']

### tranform into a dataframe

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [6]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [7]:
HUShuaineighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [8]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


### Use geopy library to get the latitude and longitude values of New York City

In [9]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### create a map of New York with neighborhoods superimposed on top

In [10]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### Analyze separately on each borough

In [11]:
print(neighborhoods['Borough'].unique())

['Bronx' 'Manhattan' 'Brooklyn' 'Queens' 'Staten Island']


#### manhattan

In [12]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [13]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [14]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [15]:
CLIENT_ID = 'FVH3QOZACD3B5VS1PADSWPGHPXT1U4JLBI0UPD0E1ZEPHRRB' # your Foursquare ID
CLIENT_SECRET = 'VEC12FLLCRUYJJ3AMIPEJXE2LAGGOHGXG3Q2ENNQCGBZYDPF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FVH3QOZACD3B5VS1PADSWPGHPXT1U4JLBI0UPD0E1ZEPHRRB
CLIENT_SECRET:VEC12FLLCRUYJJ3AMIPEJXE2LAGGOHGXG3Q2ENNQCGBZYDPF


In [18]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [19]:
# type your answer here
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)HUShuai
url



'https://api.foursquare.com/v2/venues/explore?&client_id=FVH3QOZACD3B5VS1PADSWPGHPXT1U4JLBI0UPD0E1ZEPHRRB&client_secret=VEC12FLLCRUYJJ3AMIPEJXE2LAGGOHGXG3Q2ENNQCGBZYDPF&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

In [20]:
results = requests.get(url).json()

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
HUShuaidef getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [24]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3327, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


In [25]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,100,100,100,100,100,100
Carnegie Hill,100,100,100,100,100,100
Central Harlem,45,45,45,45,45,45
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,45,45,45,45,45,45
East Village,100,100,100,100,100,100


In [26]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 340 uniques categories.


In [27]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaur

In [28]:
manhattan_onehot.shape

(3327, 341)

In [29]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaur

In [30]:
manhattan_grouped.shape

(40, 341)

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Hotel,Gym,Memorial Site,Italian Restaurant,Wine Shop,Clothing Store,Women's Store,Boat or Ferry
1,Carnegie Hill,Coffee Shop,Pizza Place,Cosmetics Shop,Café,Japanese Restaurant,Grocery Store,French Restaurant,Yoga Studio,Bookstore,Bar
2,Central Harlem,African Restaurant,Chinese Restaurant,Gym / Fitness Center,American Restaurant,French Restaurant,Cosmetics Shop,Bar,Seafood Restaurant,Public Art,Bookstore
3,Chelsea,Coffee Shop,Ice Cream Shop,Bakery,Italian Restaurant,Nightclub,Theater,American Restaurant,Seafood Restaurant,Hotel,Wine Shop
4,Chinatown,Chinese Restaurant,Cocktail Bar,Vietnamese Restaurant,Bakery,American Restaurant,Spa,Salon / Barbershop,Bubble Tea Shop,Optical Shop,Boutique


In [33]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 1, 2, 1, 2, 2, 0, 1, 1], dtype=int32)

In [34]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Sandwich Place,Coffee Shop,Yoga Studio,Deli / Bodega,Supplement Shop,Steakhouse,Seafood Restaurant,Clothing Store,Pizza Place,Department Store
1,Manhattan,Chinatown,40.715618,-73.994279,1,Chinese Restaurant,Cocktail Bar,Vietnamese Restaurant,Bakery,American Restaurant,Spa,Salon / Barbershop,Bubble Tea Shop,Optical Shop,Boutique
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Café,Bakery,Grocery Store,Mexican Restaurant,Mobile Phone Shop,Deli / Bodega,Chinese Restaurant,Pizza Place,Supermarket,Coffee Shop
3,Manhattan,Inwood,40.867684,-73.921210,0,Mexican Restaurant,Café,Lounge,Pizza Place,Spanish Restaurant,Caribbean Restaurant,Bakery,Deli / Bodega,Frozen Yogurt Shop,Chinese Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Mexican Restaurant,Café,Pizza Place,Coffee Shop,Park,Liquor Store,Indian Restaurant,Sushi Restaurant,Deli / Bodega,Sandwich Place


In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [36]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Café,Bakery,Grocery Store,Mexican Restaurant,Mobile Phone Shop,Deli / Bodega,Chinese Restaurant,Pizza Place,Supermarket,Coffee Shop
3,Inwood,Mexican Restaurant,Café,Lounge,Pizza Place,Spanish Restaurant,Caribbean Restaurant,Bakery,Deli / Bodega,Frozen Yogurt Shop,Chinese Restaurant
4,Hamilton Heights,Mexican Restaurant,Café,Pizza Place,Coffee Shop,Park,Liquor Store,Indian Restaurant,Sushi Restaurant,Deli / Bodega,Sandwich Place
5,Manhattanville,Coffee Shop,Italian Restaurant,Mexican Restaurant,Seafood Restaurant,Park,Deli / Bodega,Sushi Restaurant,Supermarket,Spanish Restaurant,Bus Station
7,East Harlem,Mexican Restaurant,Bakery,Deli / Bodega,Latin American Restaurant,Thai Restaurant,Sandwich Place,Donut Shop,Coffee Shop,Liquor Store,Beer Bar
25,Manhattan Valley,Coffee Shop,Pizza Place,Indian Restaurant,Mexican Restaurant,Thai Restaurant,Spa,French Restaurant,Deli / Bodega,Yoga Studio,Bar
36,Tudor City,Café,Park,Mexican Restaurant,Greek Restaurant,Sushi Restaurant,Deli / Bodega,Pizza Place,Garden,Dog Run,Diner


In [37]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sandwich Place,Coffee Shop,Yoga Studio,Deli / Bodega,Supplement Shop,Steakhouse,Seafood Restaurant,Clothing Store,Pizza Place,Department Store
1,Chinatown,Chinese Restaurant,Cocktail Bar,Vietnamese Restaurant,Bakery,American Restaurant,Spa,Salon / Barbershop,Bubble Tea Shop,Optical Shop,Boutique
6,Central Harlem,African Restaurant,Chinese Restaurant,Gym / Fitness Center,American Restaurant,French Restaurant,Cosmetics Shop,Bar,Seafood Restaurant,Public Art,Bookstore
9,Yorkville,Italian Restaurant,Gym,Bar,Coffee Shop,Pizza Place,Deli / Bodega,Sushi Restaurant,Japanese Restaurant,Diner,Wine Shop
10,Lenox Hill,Italian Restaurant,Coffee Shop,Pizza Place,Sushi Restaurant,Cosmetics Shop,Burger Joint,Café,Gym,Gym / Fitness Center,Sporting Goods Shop
12,Upper West Side,Italian Restaurant,Wine Bar,Coffee Shop,Indian Restaurant,Bakery,Bar,Vegetarian / Vegan Restaurant,Sushi Restaurant,Pet Store,Middle Eastern Restaurant
16,Murray Hill,Coffee Shop,Hotel,Japanese Restaurant,Sandwich Place,Italian Restaurant,French Restaurant,Gym / Fitness Center,American Restaurant,Gym,Bakery
19,East Village,Bar,Ice Cream Shop,Wine Bar,Mexican Restaurant,Pizza Place,Chinese Restaurant,Ramen Restaurant,Vegetarian / Vegan Restaurant,Cocktail Bar,Coffee Shop
20,Lower East Side,Pizza Place,Café,Coffee Shop,Chinese Restaurant,Sandwich Place,Art Gallery,Japanese Restaurant,Bakery,Ramen Restaurant,Cocktail Bar
26,Morningside Heights,Park,Bookstore,Food Truck,American Restaurant,Coffee Shop,Tennis Court,Sandwich Place,Pizza Place,New American Restaurant,Deli / Bodega


In [38]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Italian Restaurant,Exhibit,Art Gallery,Bakery,Coffee Shop,Juice Bar,French Restaurant,Gym / Fitness Center,Hotel,Spa
13,Lincoln Square,Theater,Gym / Fitness Center,Café,Plaza,Concert Hall,Italian Restaurant,French Restaurant,Park,Performing Arts Venue,Indie Movie Theater
14,Clinton,Theater,Gym / Fitness Center,American Restaurant,Spa,Hotel,Italian Restaurant,Wine Shop,Lounge,Steakhouse,Indie Theater
15,Midtown,Hotel,Clothing Store,Coffee Shop,Cocktail Bar,Theater,Bookstore,Japanese Restaurant,Café,Steakhouse,Bakery
17,Chelsea,Coffee Shop,Ice Cream Shop,Bakery,Italian Restaurant,Nightclub,Theater,American Restaurant,Seafood Restaurant,Hotel,Wine Shop
18,Greenwich Village,Italian Restaurant,Clothing Store,Sushi Restaurant,Café,Seafood Restaurant,Indian Restaurant,Cosmetics Shop,Chinese Restaurant,French Restaurant,Vietnamese Restaurant
21,Tribeca,Italian Restaurant,American Restaurant,Park,Boutique,Spa,Café,Greek Restaurant,Coffee Shop,Gym,Wine Bar
22,Little Italy,Bakery,Café,Ice Cream Shop,Sandwich Place,Salon / Barbershop,Clothing Store,Mediterranean Restaurant,Hotel,Yoga Studio,Massage Studio
23,Soho,Clothing Store,Boutique,Art Gallery,Shoe Store,Women's Store,Italian Restaurant,Sporting Goods Shop,Men's Store,Mediterranean Restaurant,Furniture / Home Store
24,West Village,Italian Restaurant,Cosmetics Shop,New American Restaurant,Park,Cocktail Bar,Jazz Club,Wine Bar,American Restaurant,Bakery,Coffee Shop


In [39]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Park,Bar,Playground,Bistro,Basketball Court,Baseball Field,Harbor / Marina,Cocktail Bar,Coffee Shop,Farmers Market


In [40]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Roosevelt Island,Sandwich Place,Coffee Shop,Park,Playground,Dry Cleaner,Bubble Tea Shop,Bridge,Farmers Market,Supermarket,Metro Station


#### Brooklyn

In [61]:
Brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
Brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [62]:
address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


In [76]:
# create map of Brooklyn using latitude and longitude values
map_Brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Brooklyn_data['Latitude'], Brooklyn_data['Longitude'],Brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Brooklyn)  
    
map_Brooklyn

In [64]:
Brooklyn_venues = getNearbyVenues(names=Brooklyn_data['Neighborhood'],
                                   latitudes=Brooklyn_data['Latitude'],
                                   longitudes=Brooklyn_data['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [65]:
print(Brooklyn_venues.shape)
Brooklyn_venues.head()

(2839, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Cocoa Grinder,40.623967,-74.030863,Juice Bar
3,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot
4,Bay Ridge,40.625801,-74.030621,Ho' Brah Taco Joint,40.622960,-74.031371,Taco Place


In [66]:
Brooklyn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,48,48,48,48,48,48
Bay Ridge,85,85,85,85,85,85
Bedford Stuyvesant,28,28,28,28,28,28
Bensonhurst,33,33,33,33,33,33
Bergen Beach,6,6,6,6,6,6
Boerum Hill,91,91,91,91,91,91
Borough Park,24,24,24,24,24,24
Brighton Beach,44,44,44,44,44,44
Broadway Junction,14,14,14,14,14,14


In [67]:
print('There are {} uniques categories.'.format(len(Brooklyn_venues['Venue Category'].unique())))

There are 288 uniques categories.


In [68]:
# one hot encoding
Brooklyn_onehot = pd.get_dummies(Brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Brooklyn_onehot['Neighborhood'] = Brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Brooklyn_onehot.columns[-1]] + list(Brooklyn_onehot.columns[:-1])
Brooklyn_onehot = Brooklyn_onehot[fixed_columns]

Brooklyn_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Fair,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky

In [69]:
Brooklyn_onehot.shape

(2839, 288)

In [70]:
Brooklyn_grouped = Brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
Brooklyn_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Fair,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky

In [71]:
Brooklyn_grouped.shape

(70, 288)

In [72]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Brooklyn_grouped['Neighborhood']

for ind in np.arange(Brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Pharmacy,Pizza Place,Fast Food Restaurant,Bubble Tea Shop,Chinese Restaurant,Sushi Restaurant,Donut Shop,Women's Store,Kids Store,Italian Restaurant
1,Bay Ridge,Spa,Italian Restaurant,Pizza Place,Bar,Grocery Store,American Restaurant,Greek Restaurant,Seafood Restaurant,Hookah Bar,Bagel Shop
2,Bedford Stuyvesant,Bar,Pizza Place,Café,Coffee Shop,Deli / Bodega,Bus Stop,Vietnamese Restaurant,BBQ Joint,Wine Shop,Fried Chicken Joint
3,Bensonhurst,Chinese Restaurant,Dessert Shop,Ice Cream Shop,Sushi Restaurant,Spa,Donut Shop,Bagel Shop,Dumpling Restaurant,Noodle House,Coffee Shop
4,Bergen Beach,Harbor / Marina,Baseball Field,Playground,Donut Shop,Athletics & Sports,Fish Market,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant


In [73]:
# set number of clusters
kclusters = 5

Brooklyn_grouped_clustering = Brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 1, 1, 0, 1, 3, 1, 1, 1], dtype=int32)

In [74]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Brooklyn_merged = Brooklyn_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Brooklyn_merged = Brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Brooklyn_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,1,Spa,Italian Restaurant,Pizza Place,Bar,Grocery Store,American Restaurant,Greek Restaurant,Seafood Restaurant,Hookah Bar,Bagel Shop
1,Brooklyn,Bensonhurst,40.611009,-73.995180,1,Chinese Restaurant,Dessert Shop,Ice Cream Shop,Sushi Restaurant,Spa,Donut Shop,Bagel Shop,Dumpling Restaurant,Noodle House,Coffee Shop
2,Brooklyn,Sunset Park,40.645103,-74.010316,3,Pizza Place,Bakery,Latin American Restaurant,Mexican Restaurant,Bank,Pharmacy,Ice Cream Shop,Deli / Bodega,Gym,Mobile Phone Shop
3,Brooklyn,Greenpoint,40.730201,-73.954241,1,Bar,Pizza Place,Coffee Shop,Cocktail Bar,French Restaurant,Mexican Restaurant,Record Shop,Café,Boutique,Yoga Studio
4,Brooklyn,Gravesend,40.595260,-73.973471,3,Italian Restaurant,Lounge,Pizza Place,Gym,Bakery,Chinese Restaurant,Diner,Martial Arts Dojo,Baseball Field,Bar


In [75]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Brooklyn_merged['Latitude'], Brooklyn_merged['Longitude'], Brooklyn_merged['Neighborhood'], Brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [77]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 0, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,Bergen Beach,Harbor / Marina,Baseball Field,Playground,Donut Shop,Athletics & Sports,Fish Market,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant


In [78]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 1, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Spa,Italian Restaurant,Pizza Place,Bar,Grocery Store,American Restaurant,Greek Restaurant,Seafood Restaurant,Hookah Bar,Bagel Shop
1,Bensonhurst,Chinese Restaurant,Dessert Shop,Ice Cream Shop,Sushi Restaurant,Spa,Donut Shop,Bagel Shop,Dumpling Restaurant,Noodle House,Coffee Shop
3,Greenpoint,Bar,Pizza Place,Coffee Shop,Cocktail Bar,French Restaurant,Mexican Restaurant,Record Shop,Café,Boutique,Yoga Studio
5,Brighton Beach,Beach,Russian Restaurant,Restaurant,Eastern European Restaurant,Gourmet Shop,Mobile Phone Shop,Sushi Restaurant,Bank,Fast Food Restaurant,Bookstore
6,Sheepshead Bay,Dessert Shop,Turkish Restaurant,Sandwich Place,Yoga Studio,Grocery Store,Department Store,Deli / Bodega,Creperie,Restaurant,Russian Restaurant
10,East Flatbush,Department Store,Food,Caribbean Restaurant,Bakery,Park,Hardware Store,Pharmacy,Supermarket,Moving Target,Deli / Bodega
11,Kensington,Grocery Store,Thai Restaurant,Sandwich Place,Pharmacy,Pizza Place,Ice Cream Shop,Liquor Store,Gas Station,Restaurant,Furniture / Home Store
12,Windsor Terrace,Park,Diner,Café,Deli / Bodega,Butcher,Sushi Restaurant,Beer Store,Coffee Shop,Grocery Store,Chinese Restaurant
13,Prospect Heights,Bar,Mexican Restaurant,Wine Shop,Ice Cream Shop,Gourmet Shop,Bakery,Thai Restaurant,Cocktail Bar,Yoga Studio,Pizza Place
15,Williamsburg,Bagel Shop,Coffee Shop,Bar,Yoga Studio,Diner,Tapas Restaurant,Latin American Restaurant,Taco Place,Steakhouse,Liquor Store


In [79]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 2, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Mill Island,Pool,Lake,Women's Store,Fish Market,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop


In [80]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 3, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Sunset Park,Pizza Place,Bakery,Latin American Restaurant,Mexican Restaurant,Bank,Pharmacy,Ice Cream Shop,Deli / Bodega,Gym,Mobile Phone Shop
4,Gravesend,Italian Restaurant,Lounge,Pizza Place,Gym,Bakery,Chinese Restaurant,Diner,Martial Arts Dojo,Baseball Field,Bar
7,Manhattan Terrace,Pizza Place,Ice Cream Shop,Donut Shop,Grocery Store,Convenience Store,Burger Joint,Mobile Phone Shop,Café,Steakhouse,Nightclub
8,Flatbush,Coffee Shop,Caribbean Restaurant,Juice Bar,Mexican Restaurant,Deli / Bodega,Bank,Pizza Place,Pharmacy,Donut Shop,Chinese Restaurant
9,Crown Heights,Pizza Place,Café,Museum,Grocery Store,Bagel Shop,Pharmacy,Park,Coffee Shop,Salon / Barbershop,Candy Store
14,Brownsville,Park,Restaurant,Playground,Plaza,Burger Joint,Fried Chicken Joint,Moving Target,Farmers Market,Caribbean Restaurant,Chinese Restaurant
25,Cypress Hills,Pizza Place,Fried Chicken Joint,Latin American Restaurant,Donut Shop,Chinese Restaurant,Ice Cream Shop,Fast Food Restaurant,Deli / Bodega,Discount Store,Coffee Shop
26,East New York,Deli / Bodega,Latin American Restaurant,Pizza Place,Event Service,Chinese Restaurant,Fast Food Restaurant,Caribbean Restaurant,Bus Station,Spanish Restaurant,Metro Station
27,Starrett City,Supermarket,Pharmacy,Shopping Mall,River,Donut Shop,Convenience Store,Caribbean Restaurant,American Restaurant,Pizza Place,Flower Shop
28,Canarsie,Gym,Asian Restaurant,Chinese Restaurant,Event Service,Caribbean Restaurant,Home Service,Thai Restaurant,Farm,Farmers Market,Fast Food Restaurant


In [81]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 4, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,Dyker Heights,Playground,Dance Studio,Burger Joint,Bagel Shop,Golf Course,Women's Store,Fish Market,Fast Food Restaurant,Field,Filipino Restaurant


#### Bronx

In [83]:
Bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
address = 'Bronx, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bronx are {}, {}.'.format(latitude, longitude))

# create map of Brooklyn using latitude and longitude values
map_Bronx = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Bronx_data['Latitude'], Bronx_data['Longitude'],Bronx_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Bronx)  
    
map_Bronx

The geograpical coordinate of Bronx are 40.85048545, -73.8404035580209.


In [86]:
Bronx_venues = getNearbyVenues(names=Bronx_data['Neighborhood'],
                                   latitudes=Bronx_data['Latitude'],
                                   longitudes=Bronx_data['Longitude']
                                  )
print(Bronx_venues.shape)
print('There are {} uniques categories.'.format(len(Bronx_venues['Venue Category'].unique())))

# one hot encoding
Bronx_onehot = pd.get_dummies(Bronx_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Bronx_onehot['Neighborhood'] = Bronx_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Bronx_onehot.columns[-1]] + list(Bronx_onehot.columns[:-1])
Bronx_onehot = Bronx_onehot[fixed_columns]

print(Bronx_onehot.shape)

Bronx_grouped = Bronx_onehot.groupby('Neighborhood').mean().reset_index()
print(Bronx_grouped.shape)

#list the top 10 common venues
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Bronx_grouped['Neighborhood']

for ind in np.arange(Bronx_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Bronx_grouped.iloc[ind, :], num_top_venues)


Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Claremont Village
Concourse Village
Mount Eden
Mount Hope
Bronxdale
Allerton
Kingsbridge Heights
(1248, 7)
There are 164 uniques categories.
(1248, 165)
(52, 165)


In [87]:
# set number of clusters
kclusters = 5

Bronx_grouped_clustering = Bronx_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Bronx_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Bronx_merged = Bronx_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Bronx_merged = Bronx_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Bronx_merged.head() # check the last columns!

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Bronx_merged['Latitude'], Bronx_merged['Longitude'], Bronx_merged['Neighborhood'], Bronx_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [88]:
Bronx_merged.loc[Bronx_merged['Cluster Labels'] == 0, Bronx_merged.columns[[1] + list(range(5, Bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Fieldston,Plaza,Bus Station,River,Playground,High School,Discount Store,Eastern European Restaurant,Dry Cleaner,Donut Shop,Historic Site


In [89]:
Bronx_merged.loc[Bronx_merged['Cluster Labels'] == 1, Bronx_merged.columns[[1] + list(range(5, Bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Woodlawn,Pizza Place,Deli / Bodega,Playground,Food Truck,Supermarket,Convenience Store,Pharmacy,Plaza,Pub,Rental Car Location
7,Norwood,Pizza Place,Park,Deli / Bodega,Bank,Mobile Phone Shop,Pharmacy,American Restaurant,Plaza,Chinese Restaurant,Supermarket
10,Pelham Parkway,Pizza Place,Italian Restaurant,Thai Restaurant,Smoke Shop,Metro Station,Sandwich Place,Bus Station,Performing Arts Venue,Mexican Restaurant,Food
11,City Island,Harbor / Marina,Park,Seafood Restaurant,Grocery Store,Thrift / Vintage Store,Bar,Pizza Place,Diner,Music Venue,Café
27,Throgs Neck,Pizza Place,Deli / Bodega,American Restaurant,Coffee Shop,Italian Restaurant,Asian Restaurant,Sports Bar,Juice Bar,Liquor Store,Bar
31,Van Nest,Pizza Place,Deli / Bodega,Coffee Shop,Supermarket,Donut Shop,BBQ Joint,Hookah Bar,Playground,Dance Studio,Fast Food Restaurant
32,Morris Park,Pizza Place,Burger Joint,Deli / Bodega,Bakery,Coffee Shop,Buffet,Sandwich Place,Donut Shop,Supermarket,Juice Bar
33,Belmont,Italian Restaurant,Deli / Bodega,Pizza Place,Bakery,Bank,Dessert Shop,Donut Shop,Spanish Restaurant,Mediterranean Restaurant,Café
35,North Riverdale,Pizza Place,Bank,Italian Restaurant,Bagel Shop,Burger Joint,Bus Station,Mexican Restaurant,Sandwich Place,Donut Shop,Sushi Restaurant
38,Edgewater Park,Italian Restaurant,Deli / Bodega,Pizza Place,Coffee Shop,Bar,Juice Bar,Food & Drink Shop,Spa,Bookstore,Donut Shop


In [90]:
Bronx_merged.loc[Bronx_merged['Cluster Labels'] == 2, Bronx_merged.columns[[1] + list(range(5, Bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Pizza Place,Dessert Shop,Food Truck,Donut Shop,Laundromat,Gas Station,Sandwich Place,Caribbean Restaurant,Pharmacy,Ice Cream Shop
2,Eastchester,Caribbean Restaurant,Metro Station,Deli / Bodega,Diner,Bus Station,Donut Shop,Bowling Alley,Food & Drink Shop,Bus Stop,Seafood Restaurant
5,Kingsbridge,Pizza Place,Sandwich Place,Bakery,Bar,Mexican Restaurant,Latin American Restaurant,Supermarket,Discount Store,Spanish Restaurant,Caribbean Restaurant
9,Baychester,Donut Shop,Pet Store,Men's Store,Electronics Store,Fast Food Restaurant,Mexican Restaurant,Spanish Restaurant,Sandwich Place,Food & Drink Shop,Supermarket
12,Bedford Park,Deli / Bodega,Diner,Pizza Place,Chinese Restaurant,Mexican Restaurant,Sandwich Place,Supermarket,Fried Chicken Joint,Park,Bakery
13,University Heights,Pizza Place,Chinese Restaurant,Deli / Bodega,Jewelry Store,Food,Sandwich Place,Donut Shop,Fried Chicken Joint,Latin American Restaurant,Supermarket
14,Morris Heights,Cosmetics Shop,Convenience Store,Pharmacy,Deli / Bodega,Spanish Restaurant,Pizza Place,Grocery Store,Food Truck,Moving Target,Latin American Restaurant
15,Fordham,Pizza Place,Shoe Store,Mobile Phone Shop,Fast Food Restaurant,Donut Shop,Bank,Gym / Fitness Center,Supplement Shop,Pharmacy,Spanish Restaurant
16,East Tremont,Pizza Place,Fast Food Restaurant,Food,Donut Shop,Spanish Restaurant,Lounge,Shoe Store,Supermarket,Mobile Phone Shop,Bank
18,High Bridge,Pharmacy,Pizza Place,Chinese Restaurant,Sandwich Place,Seafood Restaurant,Spanish Restaurant,Food,Supermarket,Market,Latin American Restaurant


In [91]:
Bronx_merged.loc[Bronx_merged['Cluster Labels'] == 3, Bronx_merged.columns[[1] + list(range(5, Bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Williamsbridge,Soup Place,Bar,Caribbean Restaurant,Nightclub,Women's Store,Dry Cleaner,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Electronics Store


In [92]:
Bronx_merged.loc[Bronx_merged['Cluster Labels'] == 4, Bronx_merged.columns[[1] + list(range(5, Bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Co-op City,Bus Station,Fried Chicken Joint,Gift Shop,Liquor Store,Baseball Field,Salon / Barbershop,Discount Store,Restaurant,Pharmacy,Mattress Store
4,Riverdale,Park,Bus Station,Plaza,Bank,Gym,Food Truck,Playground,Home Service,Art Museum,Food & Drink Shop
17,West Farms,Bus Station,Bus Stop,Playground,Donut Shop,Deli / Bodega,Lounge,Supermarket,Basketball Court,Latin American Restaurant,Bank
25,Soundview,Chinese Restaurant,Grocery Store,Playground,Latin American Restaurant,Discount Store,Bus Stop,Bus Station,Breakfast Spot,Fried Chicken Joint,Basketball Court
26,Clason Point,Park,Spa,Bus Stop,South American Restaurant,Boat or Ferry,Pool,Grocery Store,Scenic Lookout,Farmers Market,Electronics Store
34,Spuyten Duyvil,Park,Food,Tennis Stadium,Pharmacy,Intersection,Bus Line,Bank,Thai Restaurant,Fast Food Restaurant,Farmers Market
45,Claremont Village,Bus Station,Grocery Store,Bakery,Chinese Restaurant,Pizza Place,Supermarket,Liquor Store,Caribbean Restaurant,Gift Shop,Gym


#### Queens

In [139]:
Queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
address = 'Queens, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))

# create map of Brooklyn using latitude and longitude values
map_Queens = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Queens_data['Latitude'], Queens_data['Longitude'],Queens_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Queens)  
    
map_Queens

The geograpical coordinate of Queens are 40.6524927, -73.7914214158161.


In [140]:
Queens_venues = getNearbyVenues(names=Queens_data['Neighborhood'],
                                   latitudes=Queens_data['Latitude'],
                                   longitudes=Queens_data['Longitude']
                                  )
print(Queens_venues.shape)
print('There are {} uniques categories.'.format(len(Queens_venues['Venue Category'].unique())))

# one hot encoding
Queens_onehot = pd.get_dummies(Queens_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Queens_onehot['Neighborhood'] = Queens_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Queens_onehot.columns[-1]] + list(Queens_onehot.columns[:-1])
Queens_onehot = Queens_onehot[fixed_columns]

print(Queens_onehot.shape)

Queens_grouped = Queens_onehot.groupby('Neighborhood').mean().reset_index()
print(Queens_grouped.shape)

#list the top 10 common venues
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Queens_grouped['Neighborhood']

for ind in np.arange(Queens_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Queens_grouped.iloc[ind, :], num_top_venues)



Astoria
Woodside
Jackson Heights
Elmhurst
Howard Beach
Corona
Forest Hills
Kew Gardens
Richmond Hill
Flushing
Long Island City
Sunnyside
East Elmhurst
Maspeth
Ridgewood
Glendale
Rego Park
Woodhaven
Ozone Park
South Ozone Park
College Point
Whitestone
Bayside
Auburndale
Little Neck
Douglaston
Glen Oaks
Bellerose
Kew Gardens Hills
Fresh Meadows
Briarwood
Jamaica Center
Oakland Gardens
Queens Village
Hollis
South Jamaica
St. Albans
Rochdale
Springfield Gardens
Cambria Heights
Rosedale
Far Rockaway
Broad Channel
Breezy Point
Steinway
Beechhurst
Bay Terrace
Edgemere
Arverne
Rockaway Beach
Neponsit
Murray Hill
Floral Park
Holliswood
Jamaica Estates
Queensboro Hill
Hillcrest
Ravenswood
Lindenwood
Laurelton
Lefrak City
Belle Harbor
Rockaway Park
Somerville
Brookville
Bellaire
North Corona
Forest Hills Gardens
Jamaica Hills
Utopia
Pomonok
Astoria Heights
Hunters Point
Sunnyside Gardens
Blissville
Roxbury
Middle Village
Malba
Hammels
Bayswater
Queensbridge
(2151, 7)
There are 270 uniques categor

In [141]:
# set number of clusters
kclusters = 5

Queens_grouped_clustering = Queens_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Queens_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Queens_merged = Queens_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Queens_merged = Queens_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Queens_merged.head() # check the last columns!

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Queens_merged['Latitude'], Queens_merged['Longitude'], Queens_merged['Neighborhood'], Queens_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [102]:
Queens_merged.loc[Queens_merged['Cluster Labels'] == 0, Queens_merged.columns[[1] + list(range(5, Queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Somerville,Park,Women's Store,Eye Doctor,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Dry Cleaner
79,Bayswater,Playground,Park,Women's Store,Eye Doctor,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant


In [103]:
Queens_merged.loc[Queens_merged['Cluster Labels'] == 1, Queens_merged.columns[[1] + list(range(5, Queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Astoria,Middle Eastern Restaurant,Bar,Greek Restaurant,Hookah Bar,Seafood Restaurant,Bakery,Mediterranean Restaurant,Gym,Food Truck,Italian Restaurant
1,Woodside,Grocery Store,Thai Restaurant,Latin American Restaurant,Pizza Place,Bakery,Filipino Restaurant,Donut Shop,Bar,American Restaurant,Pub
2,Jackson Heights,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Mobile Phone Shop,Thai Restaurant,Diner,Mexican Restaurant,Supplement Shop,Spanish Restaurant
3,Elmhurst,Thai Restaurant,Mexican Restaurant,South American Restaurant,Chinese Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Malay Restaurant,Salon / Barbershop,Sushi Restaurant,Donut Shop
4,Howard Beach,Italian Restaurant,Pharmacy,Fast Food Restaurant,Gym,Sandwich Place,Bagel Shop,Spa,Flower Shop,Fried Chicken Joint,Sushi Restaurant
5,Corona,Mexican Restaurant,Italian Restaurant,South American Restaurant,Convenience Store,Empanada Restaurant,Supermarket,Sandwich Place,Donut Shop,Chinese Restaurant,Restaurant
6,Forest Hills,Gym,Gym / Fitness Center,Yoga Studio,Pharmacy,Pizza Place,Park,Food Truck,Thai Restaurant,Convenience Store,Snack Place
7,Kew Gardens,Chinese Restaurant,Park,Cosmetics Shop,Bar,Deli / Bodega,Donut Shop,Bank,Indian Restaurant,Pizza Place,Juice Bar
8,Richmond Hill,Pizza Place,Latin American Restaurant,Caribbean Restaurant,Bank,Lounge,Diner,Sandwich Place,Chinese Restaurant,Moving Target,Metro Station
9,Flushing,Chinese Restaurant,Hotpot Restaurant,Korean Restaurant,Bubble Tea Shop,Spa,Construction & Landscaping,Asian Restaurant,Karaoke Bar,Szechuan Restaurant,Cantonese Restaurant


In [104]:
Queens_merged.loc[Queens_merged['Cluster Labels'] == 2, Queens_merged.columns[[1] + list(range(5, Queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,Brookville,Deli / Bodega,Women's Store,Falafel Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space,Eye Doctor,Farm


In [105]:
Queens_merged.loc[Queens_merged['Cluster Labels'] == 3, Queens_merged.columns[[1] + list(range(5, Queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,Laurelton,Caribbean Restaurant,Train Station,Cosmetics Shop,Cuban Restaurant,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market


In [106]:
Queens_merged.loc[Queens_merged['Cluster Labels'] == 4, Queens_merged.columns[[1] + list(range(5, Queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,Breezy Point,American Restaurant,Board Shop,Monument / Landmark,Beach,Trail,Women's Store,Falafel Restaurant,Electronics Store,Empanada Restaurant,Event Space
50,Neponsit,Beach,Bus Stop,Women's Store,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Food
61,Belle Harbor,Beach,Deli / Bodega,Spa,Bus Stop,Chinese Restaurant,Bakery,Bagel Shop,Donut Shop,Italian Restaurant,Boutique
62,Rockaway Park,Beach,Pizza Place,Donut Shop,Bank,Diner,Smoke Shop,Seafood Restaurant,Liquor Store,Board Shop,Steakhouse
75,Roxbury,Beach,Pub,Fast Food Restaurant,Deli / Bodega,Baseball Field,Irish Pub,Trail,Women's Store,Eye Doctor,Egyptian Restaurant
78,Hammels,Beach,Bus Station,Building,Fast Food Restaurant,Deli / Bodega,Southern / Soul Food Restaurant,Café,Food Truck,Diner,Shoe Store


#### Staten_Island

In [166]:
Staten_Island_data = neighborhoods[neighborhoods['Borough'] == 'Staten Island'].reset_index(drop=True)
address = 'Staten Island, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Staten Island are {}, {}.'.format(latitude, longitude))

# create map of Brooklyn using latitude and longitude values
map_StatenIsland = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Staten_Island_data['Latitude'], Staten_Island_data['Longitude'],Staten_Island_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_StatenIsland)  
    
map_StatenIsland

The geograpical coordinate of Staten Island are 40.5834557, -74.1496048.


In [167]:
Staten_Island_venues = getNearbyVenues(names=Staten_Island_data['Neighborhood'],
                                   latitudes=Staten_Island_data['Latitude'],
                                   longitudes=Staten_Island_data['Longitude']
                                  )
print(Staten_Island_venues.shape)
print('There are {} uniques categories.'.format(len(Staten_Island_venues['Venue Category'].unique())))

# one hot encoding
Staten_Island_onehot = pd.get_dummies(Staten_Island_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Staten_Island_onehot['Neighborhood'] = Staten_Island_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Staten_Island_onehot.columns[-1]] + list(Staten_Island_onehot.columns[:-1])
Staten_Island_onehot = Staten_Island_onehot[fixed_columns]

print(Staten_Island_onehot.shape)

Staten_Island_grouped = Staten_Island_onehot.groupby('Neighborhood').mean().reset_index()
print(Staten_Island_grouped.shape)

#list the top 10 common venues
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Staten_Island_grouped['Neighborhood']

for ind in np.arange(Staten_Island_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Staten_Island_grouped.iloc[ind, :], num_top_venues)


St. George
New Brighton
Stapleton
Rosebank
West Brighton
Grymes Hill
Todt Hill
South Beach
Port Richmond
Mariner's Harbor
Port Ivory
Castleton Corners
New Springville
Travis
New Dorp
Oakwood
Great Kills
Eltingville
Annadale
Woodrow
Tottenville
Tompkinsville
Silver Lake
Sunnyside
Park Hill
Westerleigh
Graniteville
Arlington
Arrochar
Grasmere
Old Town
Dongan Hills
Midland Beach
Grant City
New Dorp Beach
Bay Terrace
Huguenot
Pleasant Plains
Butler Manor
Charleston
Rossville
Arden Heights
Greenridge
Heartland Village
Chelsea
Bloomfield
Bulls Head
Richmond Town
Shore Acres
Clifton
Concord
Emerson Hill
Randall Manor
Howland Hook
Elm Park
Manor Heights
Willowbrook
Sandy Ground
Egbertville
Prince's Bay
Lighthouse Hill
Richmond Valley
Fox Hills
(844, 7)
There are 182 uniques categories.
(844, 183)
(61, 183)


In [168]:
# set number of clusters
kclusters = 5

Staten_Island_grouped_clustering = Staten_Island_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Staten_Island_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Staten_Island_merged = Staten_Island_data

Staten_Island_merged = Staten_Island_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Staten_Island_merged = Staten_Island_merged.dropna(how='any') 
Staten_Island_merged['Cluster Labels'] = Staten_Island_merged['Cluster Labels'].astype(int)

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Staten_Island_merged['Latitude'], Staten_Island_merged['Longitude'], Staten_Island_merged['Neighborhood'], Staten_Island_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [169]:
Staten_Island_merged.loc[Staten_Island_merged['Cluster Labels'] == 0, Staten_Island_merged.columns[[1] + list(range(5, Staten_Island_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Stapleton,Mexican Restaurant,Sandwich Place,Discount Store,Bank,Pizza Place,Italian Restaurant,Motorcycle Shop,Spanish Restaurant,Skate Park,Breakfast Spot
3,Rosebank,Pizza Place,Italian Restaurant,Grocery Store,Eastern European Restaurant,Mexican Restaurant,Breakfast Spot,Filipino Restaurant,Museum,Cajun / Creole Restaurant,Donut Shop
4,West Brighton,Coffee Shop,Bank,Pharmacy,American Restaurant,Cosmetics Shop,Music Store,Italian Restaurant,Bar,Ice Cream Shop,Mexican Restaurant
8,Port Richmond,Rental Car Location,Automotive Shop,Bus Station,Basketball Court,Martial Arts Dojo,Donut Shop,Breakfast Spot,Pizza Place,Construction & Landscaping,Diner
11,Castleton Corners,Pizza Place,Ice Cream Shop,Sandwich Place,Mini Golf,Go Kart Track,Tattoo Parlor,Bar,Bank,Bakery,Bagel Shop
12,New Springville,Bagel Shop,Pizza Place,Ice Cream Shop,Hookah Bar,Sandwich Place,Restaurant,Pharmacy,Optical Shop,Mobile Phone Shop,Mexican Restaurant
13,Travis,Hotel,Deli / Bodega,Baseball Field,Pizza Place,Comedy Club,Donut Shop,Park,Business Service,Spanish Restaurant,Sports Club
16,Great Kills,Pizza Place,Bar,Italian Restaurant,Cosmetics Shop,Japanese Restaurant,Food & Drink Shop,Falafel Restaurant,Mexican Restaurant,Liquor Store,Spanish Restaurant
17,Eltingville,Pizza Place,Sushi Restaurant,Italian Restaurant,Pharmacy,Bank,Liquor Store,Fast Food Restaurant,Diner,Donut Shop,Sandwich Place
18,Annadale,Pub,Train Station,Diner,Sports Bar,Bagel Shop,Pharmacy,Restaurant,Bakery,Deli / Bodega,Dance Studio


In [170]:
Staten_Island_merged.loc[Staten_Island_merged['Cluster Labels'] == 1, Staten_Island_merged.columns[[1] + list(range(5, Staten_Island_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Grymes Hill,Dog Run,Yoga Studio,Eastern European Restaurant,Food Truck,Food & Drink Shop,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant


In [171]:
Staten_Island_merged.loc[Staten_Island_merged['Cluster Labels'] == 2, Staten_Island_merged.columns[[1] + list(range(5, Staten_Island_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,New Brighton,Bus Stop,Deli / Bodega,Park,Convenience Store,Home Service,Playground,Discount Store,Bowling Alley,Event Space,Food & Drink Shop
7,South Beach,Beach,Pier,Deli / Bodega,Athletics & Sports,Yoga Studio,Event Space,Food Truck,Food & Drink Shop,Food,Flower Shop
15,Oakwood,Playground,Bus Stop,Bar,Yoga Studio,Eastern European Restaurant,Food & Drink Shop,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant
24,Park Hill,Bus Stop,Hotel,Coffee Shop,Athletics & Sports,Gym / Fitness Center,Dim Sum Restaurant,Event Space,Food & Drink Shop,Food,Flower Shop
27,Arlington,Bus Stop,American Restaurant,Deli / Bodega,Intersection,Boat or Ferry,Yoga Studio,Event Space,Food Truck,Food & Drink Shop,Food
29,Grasmere,Bus Stop,Bagel Shop,Bank,Grocery Store,Ice Cream Shop,Cosmetics Shop,Nail Salon,Park,Deli / Bodega,Juice Bar
32,Midland Beach,Deli / Bodega,Bus Stop,Beach,Dessert Shop,Restaurant,Bookstore,Pet Store,Liquor Store,Basketball Court,Falafel Restaurant
38,Butler Manor,Pool,Baseball Field,Convenience Store,Bus Stop,Eastern European Restaurant,Food & Drink Shop,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant
45,Bloomfield,Recreation Center,Theme Park,Bus Stop,Discount Store,Park,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
56,Willowbrook,Bus Stop,Deli / Bodega,Pizza Place,Chinese Restaurant,Bagel Shop,Yoga Studio,Eastern European Restaurant,Food & Drink Shop,Food,Flower Shop


In [172]:
Staten_Island_merged.loc[Staten_Island_merged['Cluster Labels'] == 3, Staten_Island_merged.columns[[1] + list(range(5, Staten_Island_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,St. George,Clothing Store,Bar,American Restaurant,Park,Italian Restaurant,Coffee Shop,Monument / Landmark,Scenic Lookout,Sandwich Place,Donut Shop
9,Mariner's Harbor,Deli / Bodega,Italian Restaurant,Bus Stop,Other Repair Shop,Yoga Studio,Event Space,Food & Drink Shop,Food,Flower Shop,Filipino Restaurant
14,New Dorp,Italian Restaurant,Mexican Restaurant,Pizza Place,Coffee Shop,Bank,Indian Restaurant,Gas Station,Dim Sum Restaurant,Dessert Shop,Deli / Bodega
20,Tottenville,Italian Restaurant,Cosmetics Shop,Bus Stop,Mexican Restaurant,Park,Thrift / Vintage Store,Frame Store,Deli / Bodega,Home Service,Event Space
23,Sunnyside,Theater,Market,American Restaurant,Spa,Gym,Grocery Store,Donut Shop,Food,Flower Shop,Filipino Restaurant
25,Westerleigh,Convenience Store,Bank,Arcade,Basketball Court,Frame Store,Food Truck,Food & Drink Shop,Food,Flower Shop,Filipino Restaurant
26,Graniteville,Supermarket,Boat or Ferry,Moving Target,Deli / Bodega,Falafel Restaurant,Dance Studio,Frame Store,Food Truck,Food & Drink Shop,Food
28,Arrochar,Bus Stop,Deli / Bodega,Italian Restaurant,Bagel Shop,Food Truck,Supermarket,Mediterranean Restaurant,Taco Place,Liquor Store,Sandwich Place
30,Old Town,Italian Restaurant,Pizza Place,Middle Eastern Restaurant,Mattress Store,Donut Shop,Liquor Store,Optical Shop,Jewelry Store,Bank,Bakery
34,New Dorp Beach,Deli / Bodega,Italian Restaurant,Sports Bar,Restaurant,Scenic Lookout,Diner,Food,Beach,Skating Rink,Dry Cleaner


In [173]:
Staten_Island_merged.loc[Staten_Island_merged['Cluster Labels'] == 4, Staten_Island_merged.columns[[1] + list(range(5, Staten_Island_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Todt Hill,Trail,Park,Yoga Studio,Dry Cleaner,Food & Drink Shop,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
52,Randall Manor,Park,Bus Stop,Pizza Place,Carpet Store,Yoga Studio,Eastern European Restaurant,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant


# Discussion and conclusions